# BAT.jl: Background + Signal Analysis 

***
In this exercise, we will get familiar with fitting a model to datasets of events, measured from your experiment, which are supplied as histograms. 

You will have access to two different datasets, which you will use separately to construct knowledge of the background of your experiment and later on to look for a signal using a Bayesian approach. The first dataset contains a large population of events and this is the one you will use to perform a Bayesian fit of the model for the background of your experiment. 

After you have obtained a model of the background, you can use the second dataset, which has a low event population, to test whether a signal is present by computing the evidence for both the background model only and the background model with a superimposed signal. 

Given the evidence for both hypotheses, you will decide whether a signal is present using the  [Bayes factor](https://en.wikipedia.org/wiki/Bayes_factor).

***


To start we need to import some packages that we will need for our analysis:

In [ ]:
using Random, LinearAlgebra, Statistics, Distributions, StatsBase
using EponymTuples
using BAT, IntervalSets
using ValueShapes
using Plots
pyplot(palette = :inferno, size=(750,500))

## 1. Generate Data

In [ ]:
bins=0:0.5:30
N_background = 1500
λ = 10

data_background = rand(Truncated(Exponential(λ), 0, 30), N_background)
hist_background = append!(Histogram(bins), data_background);

In [ ]:
N_signal = 43
N_s_background = 400
μ_true = 15.0
σ_true = 1.0

data_signal =  vcat(
    rand(Truncated(Exponential(λ), 0, 30), N_s_background),
    rand(Normal( μ_true, σ_true), N_signal)
)

hist_signal = append!(Histogram(bins), data_signal);

In [ ]:
plot(normalize(hist_background, mode=:density), alpha=0.9, color=1, fillalpha = 0.05, linewidth = 0.0, label="")
plot!(normalize(hist_background, mode=:density), seriestype = :steps, alpha=0.9, color=1,  label="Background Data")
plot!(normalize(hist_signal, mode=:density), alpha=0.9, color=3, label="Signal Data")
plot!(xlabel = "x", ylabel="counts", title="Histogram of the Data")

## 2. Definition of Model Functions & Likelihood

Given the data histograms, we have to declare the model functions we want to fit. 

Below you can see how a second degree polynomial is defined as an example. You can notice that you can use both a scalar ($\lambda$) or a vector of variables ($A$) to define the parameters of your model. 

Now you can define you models for the background and for the signal. Given the background histogram we think that an Exponential funciton would be a good candidate. For the signal model you may use a simpla Gaussian distribution.

In [ ]:
function background_model(@eponymargs(A, λ), x::Real)
    A[1] + A[2]*exp(-λ*x)
end

function background_signal_model(@eponymargs(A, λ, D, x_0, σ), x::Real)
    A[1] + A[2]*exp(-λ*x) + (D/(sqrt(2*pi*σ^2)))*exp(-((x - x_0)^2)/σ^2)  
end

First, we need to define a likelihood function for our problem. In BAT, all likelihood functions and priors are subtypes of BAT.AbstractDensity. We'll store the histogram that we want to fit in our likelihood density type, as accessing the histogram as a global variable would reduce performance:

In [ ]:
struct HistogramLikelihood{H<:Histogram,F<:Function} <: AbstractDensity
    histogram::H
    fitfunc::F
end

In [ ]:
function BAT.density_logval(
    likelihood::HistogramLikelihood,
    params::Union{NamedTuple,AbstractVector{<:Real}}
)
    # Histogram counts for each bin as an array:
    counts = likelihood.histogram.weights

    # Histogram binning, has length (length(counts) + 1):
    binning = likelihood.histogram.edges[1]
    
    log_likelihood::Float64 = 0.0
    for i in eachindex(counts)
        bin_left, bin_right = binning[i], binning[i+1]
        bin_width = bin_right - bin_left
        bin_center = (bin_right + bin_left) / 2
        observed_counts = counts[i]
        expected_counts = bin_width * likelihood.fitfunc(params, bin_center)
        log_likelihood += logpdf(Poisson(expected_counts), observed_counts) 
    end

    return log_likelihood
end

In [ ]:
algorithm = MetropolisHastings()
nsamples = 2*10^5
nchains = 5
ENV["JULIA_INFO"] = "BAT";

## 3. Background Analysis: 

Now we'll perform a Bayesian fit of the background histogram, using BAT, to infer the model parameters from the data histogram.

In [ ]:
likelihood_bm = HistogramLikelihood(hist_background, background_model);

We need to choose a sensible prior for the fit:

In [ ]:
prior_bm = NamedPrior(
    A = [0.0 .. 1000.0, 0 .. 10000.0],
    λ = 0.0 .. 20.0
);

In [ ]:
parshapes_bm = valshape(prior_bm)
posterior_bm = PosteriorDensity(likelihood_bm, prior_bm);

In [ ]:
samples_bm, sampleids_bm, stats_bm, chains_bm = rand(MCMCSpec(algorithm, posterior_bm), nsamples, nchains);

In [ ]:
println("Mode: $(stats_bm.mode)")
println("Mean: $(stats_bm.param_stats.mean)")

In [ ]:
mode_parms_bm = parshapes_bm(stats_bm.mode)

In [ ]:
plot(samples_bm, params=[1,2,3], param_labels=["A_1","A_2","\$\\lambda\$"])

In [ ]:
plot(normalize(hist_background, mode=:density), alpha=0.9, color=1, fillalpha = 0.05, linewidth = 0.0, label="")
plot!(normalize(hist_background, mode=:density), seriestype = :steps, alpha=0.9, color=1,  label="Background Data")

plot!(xlabel = "x", ylabel="counts", title="Background Analysis")

plot!(
    0:0.1:30, x -> background_model(mode_parms_bm, x),
    label = "Background Model", lw=1.5, color=1,  linestyle = :dot
)


### Histogram as a prior: 

We now use posterior obtained above as a prior for signal+data fit

In [ ]:
samples = hcat(samples_bm.params...)[3,:]

λ_hist = fit(Histogram, samples, FrequencyWeights(samples_bm.weight), nbins = 150)

## 4. Background + Signal Analysis: 

Let us use the second dataset, which has much fewer samples, to infer parameters of the signal model 

In [ ]:
likelihood_signal = HistogramLikelihood(hist_signal, background_signal_model);

prior_signal = NamedPrior(
    A = [0.0 .. 1000.0, 0 .. 10000.0],
    λ = BAT.HistogramAsUvDistribution(λ_hist),
    D = 0.0..1000, 
    x_0 = 0 .. 30, 
    σ = 0.5 .. 10.0
);

parshapes_signal = valshape(prior_signal)
posterior_signal = PosteriorDensity(likelihood_signal, prior_signal);

In [ ]:
samples_signal, sampleids_signal, stats_signal, chains_signal = rand(MCMCSpec(algorithm, posterior_signal), nsamples, nchains);

In [ ]:
plot(samples_signal, params=[1,2,3], param_labels=["A_1","A_2","\$\\lambda\$"])

In [ ]:
plot(samples_signal, params=[4,5,6], param_labels=["D","x_0","\$\\sigma\$"])

In [ ]:
plot(samples_signal, 3)
plot!(prior_signal, 3)
plot!(ylabel="P(λ)", xlabel="λ", title="Knowledge Update")

In [ ]:
mode_parms_signal = parshapes_signal(stats_signal.mode)

In [ ]:
plot(normalize(hist_signal, mode=:density), fillalpha=0.65, color=3, label="Signal Data")
plot!(
    0:0.1:30, x -> background_signal_model(mode_parms_signal, x),
    label = "Signal Model", lw=2, color=6, linestyle = :dash
)
plot!(xlabel = "x", ylabel="counts", title="Signal Analysis")

The Bayes factor for the signal model can be computed in the following way

In [ ]:
Z_signal = HMIData(samples_signal)

hm_integrate!(Z_signal)

## 5. Signal vs. No signal: 

Let us now analyze the same dataset under the assumption that we have no signal events

In [ ]:
likelihood_nosignal = HistogramLikelihood(hist_signal, background_model);

prior_nosignal = NamedPrior(
    A = [0.0 .. 1000.0, 0 .. 10000.0],
    λ = BAT.HistogramAsUvDistribution(λ_hist),
);


parshapes_nosignal = valshape(prior_nosignal)
posterior_nosignal = PosteriorDensity(likelihood_nosignal, prior_nosignal);

In [ ]:
samples_nosignal, sampleids_nosignal, stats_nosignal, chains_nosignal = rand(MCMCSpec(algorithm, posterior_nosignal), nsamples, nchains);

In [ ]:
mode_parms_nosignal = parshapes_nosignal(stats_nosignal.mode)

In [ ]:
plot(normalize(hist_signal, mode=:density), fillalpha=0.65, color=3, label="Signal Data")
plot!(
    0:0.1:30, x -> background_model(mode_parms_nosignal, x),
    label = "Background Model", lw=2, color=1,  linestyle = :dot
)
plot!(
    0:0.1:30, x -> background_signal_model(mode_parms_signal, x),
    label = "Signal Model", lw=2, color=6, linestyle = :dash
)
plot!(xlabel = "x", ylabel="counts", title="Signal Analysis")

The Bayes factor for this case is equal to

In [ ]:
Z_nosignal = HMIData(samples_nosignal)
        
hm_integrate!(Z_nosignal)

Finally, let us compare Bayes factors for these two assumptions: 

In [ ]:
Z_signal_v = Z_signal.integralestimates["analytic result"].final.estimate
Z_nosignal_v = Z_nosignal.integralestimates["analytic result"].final.estimate;

@show Z_signal_v/Z_nosignal_v;